# requests

> This module provides a wrapper for interating with SQLite Database(Chosen datastore for qgofer)

In [ ]:
# | default_exp manage_index

In [ ]:
# | hide
import nbdev
from nbdev.showdoc import *

In [ ]:
# | export
import os
from asyncio import run as async_run
from pathlib import Path
from typing import Any, AsyncGenerator, Generator


import aiofiles

try:
    from qgoferutils.core import folders_to_index, VIDEO, AUDIO, IMAGE
    from qgoferutils.config import QGoferConfig
    from qgoferutils.db import QGoferDBWrapper
except:
    pass


In [ ]:
%run 00_core.ipynb
%run 03_config.ipynb
%run 01_db.ipynb

2024-03-07 22:18:02,267 - qgofer - INFO - Logging to C:\Users\acqua\.qgofer\logs/913G7k-OQIWfXAZnpF8b7g_log_24_03_07_22_18_02.log
IntegrityError occurred: UNIQUE constraint failed: qgofer_config.key


In [ ]:
# | export

Q = QGoferConfig()
CACHE_TABLE = "qgofer_cache"
INDEX_LIST = "inde_list.txt"

In [ ]:
!pip install aiofiles

In [ ]:
# | export


def index_folders(
    path: Path, data_type: str = "document"
) -> Generator[Path, None, None]:
    """Index the folder."""
    path = Path(path).expanduser().resolve()
    try:
        for item in path.iterdir():
            if item.name.startswith("."):
                continue
            if item.is_dir():
                if item.name in folders_to_index:
                    yield from index_folders(item, data_type)
            elif item.is_file():
                if data_type == "document":
                    if item.suffix not in (VIDEO + AUDIO + IMAGE):
                        yield item
                elif data_type == "video":
                    if item.suffix in VIDEO:
                        yield item
                elif data_type == "audio":
                    if item.suffix in AUDIO:
                        yield item
                elif data_type == "image":
                    if item.suffix in IMAGE:
                        yield item
                else:
                    Q.logger.info(f"Item with suffix: {item.suffix} is not supported.")
    except (PermissionError, FileNotFoundError) as err:
        Q.logger.error(err)

In [ ]:
# | export


def create_index_list(path: Path) -> None:
    """Index the folders."""
    # A list to be populated with async tasks.
    if path:
        index_path = path
    else:
        index_path = Q.root_dir
    for item in index_folders(index_path):
        with open(Q.qgofer_cache / INDEX_LIST, "a", encoding="utf-8") as f:
            f.write(f"{item}\n")

In [ ]:
# | export


def create_qgofer_cache() -> None:
    """Create the qgofer documents cache database."""
    qgofer_db_wrapper = QGoferDBWrapper(Q.qgofer_cache_db)
    qgofer_db_wrapper.create_table(
        f"{CACHE_TABLE}",
        "a_time REAL,m_time REAL,c_time REAL,file_size REAL,file TEXT UNIQUE",
    )


def clear_cache_table() -> None:
    """Clear the qgofer cache database."""
    qgofer_db_wrapper = QGoferDBWrapper(Q.qgofer_cache_db)
    qgofer_db_wrapper.drop_table(f"{CACHE_TABLE}")

In [ ]:
# | export


# TODO: Clean up this function
def process_index_list():
    """Process the index list."""

    async def _read_index_list() -> AsyncGenerator[Path, None]:
        """Read the index list."""
        async with aiofiles.open(
            Q.qgofer_cache / "inde_list.txt", "r", encoding="utf-8"
        ) as f:
            async for line in f:
                yield Path(line.strip())

    async def _process_files() -> AsyncGenerator[tuple[str, Path], None]:
        """Process the index list."""
        async for item in _read_index_list():
            yield item

    async def _gen_meta() -> (
        AsyncGenerator[tuple[Any, float, float, float, int, Path], None]
    ):
        """Generate the meta data."""
        async for file in _process_files():
            a_time = os.path.getatime(file)
            m_time = os.path.getmtime(file)
            c_time = os.path.getctime(file)
            file_size = os.path.getsize(file)
            yield a_time, m_time, c_time, file_size, file

    async def _process_meta() -> None:
        """Process the meta data."""
        async for a_time, m_time, c_time, file_size, file in _gen_meta():
            qgofer_db_wrapper = QGoferDBWrapper(Q.qgofer_cache_db)

            try:
                qgofer_db_wrapper.insert(
                    CACHE_TABLE,
                    "a_time, m_time, c_time, file_size, file",
                    f"{a_time}, {m_time}, {c_time}, {file_size}, '{file}'",
                )
            except Exception as err:
                Q.logger.error(err)
            # need to save file name and other meta data at least
            Q.logger.info("************ Indexing Meta Data ************")

    async_run(_process_meta())

In [ ]:
# | export


def destroy_index_list() -> None:
    try:
        os.remove(Q.qgofer_cache / INDEX_LIST)
    except OSError:
        pass

In [ ]:
create_index_list(r"C:\Users\acqua\OneDrive\Documents\Books")

In [ ]:
create_qgofer_cache()

In [ ]:
destroy_index_list()

In [ ]:
#| hide
nbdev.nbdev_export()